In [1]:
# Google Colaboratoryを使用する場合は、下記のコメントアウトを外し、実行してください

In [2]:
# # 下記セルを実行すると、authorization codeの入力を求められます。
# # 出力されたリンク先をクリックし、Googleアカウントにログインし、
# # authorization codeをコピーし、貼り付けをおこなってください。
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# working_dir = 'MLSys_100Knocks' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
# path = f'/content/drive/MyDrive/{working_dir}/本章/3章'
# os.chdir(path)
# ! pip install japanize-matplotlib

# ３章　可視化の仕組みを構築する１０本ノック

### ノック２１：店舗を絞り込んで可視化できるようにしてみよう

In [4]:
import pandas as pd
from IPython.display import display, clear_output

m_store = pd.read_csv('m_store.csv')
m_area = pd.read_csv('m_area.csv')
order_data = pd.read_csv('tbl_order_202004.csv')
order_data = pd.merge(order_data, m_store, on='store_id', how='left')
order_data = pd.merge(order_data, m_area, on='area_cd', how='left')

# マスターにないコードに対応した文字列を設定
order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'デリバリー'
order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'お持ち帰り'

order_data.loc[order_data['status'] == 0, 'status_name'] = '受付'
order_data.loc[order_data['status'] == 1, 'status_name'] = 'お支払済'
order_data.loc[order_data['status'] == 2, 'status_name'] = 'お渡し済'
order_data.loc[order_data['status'] == 9, 'status_name'] = 'キャンセル'

order_data.head()

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name
0,79339111,49,C26387220,50,67393872,2020-04-01 11:00:00,2020-04-01 11:18:00,1,4144,1,浅草店,TK,東京,東京,お持ち帰り,お支払済
1,18941733,85,C48773811,26,91834983,2020-04-01 11:00:00,2020-04-01 11:22:00,0,2877,2,目黒店,TK,東京,東京,デリバリー,お渡し済
2,56217880,76,C66287421,36,64409634,2020-04-01 11:00:00,2020-04-01 11:15:00,0,2603,2,本郷店,TK,東京,東京,デリバリー,お渡し済
3,28447783,190,C41156423,19,73032165,2020-04-01 11:00:00,2020-04-01 11:16:00,0,2732,2,栃木店,TO,北関東,栃木,デリバリー,お渡し済
4,32576156,191,C54568117,71,23281182,2020-04-01 11:00:00,2020-04-01 11:53:00,0,2987,2,伊勢崎店,GU,北関東,群馬,デリバリー,お渡し済


In [5]:
from ipywidgets import Dropdown

def order_by_store(val):
    clear_output()
    display(dropdown)    
    pick_data = order_data.loc[(order_data['store_name']==val['new']) & (order_data['status'].isin([1, 2]))]
    display(pick_data.head())

store_list = m_store['store_name'].tolist()

dropdown = Dropdown(options=store_list)
dropdown.observe(order_by_store, names='value')
display(dropdown)

Dropdown(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛西店', '…

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import japanize_matplotlib

def graph_by_store(val):
    clear_output()
    display(dropdown2)
    pick_data = order_data.loc[(order_data['store_name']==val['new']) & (order_data['status'].isin([1, 2]))]
    temp = pick_data[['order_accept_date', 'total_amount']].copy()  
    temp.loc[:,'order_accept_date'] = pd.to_datetime(temp['order_accept_date'])
    temp.set_index('order_accept_date', inplace=True)
    temp.resample('D').sum().plot()
    
dropdown2 = Dropdown(options=store_list)
dropdown2.observe(graph_by_store, names='value')
display(dropdown2)

Dropdown(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛西店', '…

### ノック２２：複数店舗の詳細を可視化できるようにしてみよう

In [7]:
from ipywidgets import SelectMultiple

def order_by_multi(val):
    clear_output()
    display(select)
    pick_data = order_data.loc[(order_data["store_name"].isin(val["new"])) & (order_data["status"].isin([1, 2]))]
    display(pick_data.head())

select = SelectMultiple(options=store_list)
select.observe(order_by_multi, names="value")
display(select)

SelectMultiple(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛…

In [8]:
def graph_by_multi(val):
    clear_output()
    display(select2)

    fig = plt.figure(figsize=(17,4))
    plt.subplots_adjust(wspace=0.25, hspace=0.6)

    i=0

    for trg in val["new"]:
        pick_data = order_data[(order_data["store_name"]==trg) & (order_data["status"].isin([1,2]))]
        temp = pick_data[["order_accept_date", "total_amount", "store_name"]].copy()
        temp.loc[:, "order_accept_date"] = pd.to_datetime(temp["order_accept_date"])
        temp.set_index("order_accept_date", inplace=True)
        i += 1
        ax = fig.add_subplot(1, len(val["new"]), i)
        ax.plot(temp.resample("D").sum())
        ax.set_title(trg)

select2 = SelectMultiple(options=store_list)
select2.observe(graph_by_multi, names="value")
display(select2) 

SelectMultiple(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛…

### ノック２３：スライドバーを用いてオーダー件数を調べてみよう

In [9]:
from ipywidgets import IntSlider

def store_lower(val):
    clear_output()
    display(slider)
    temp = order_data.groupby("store_name")
    print(temp.size()[temp.size() < val["new"]])

slider = IntSlider(value=1100, min=1000, max=2000, step=100, description="件数:",)
slider.observe(store_lower, names="value")
display(slider)

IntSlider(value=1100, description='件数:', max=2000, min=1000, step=100)

In [10]:
def store_upper(val):
    clear_output()
    display(slider2)
    temp = order_data.groupby("store_name")
    print(temp.size()[temp.size() > val["new"]])

slider2 = IntSlider(value=1600, min=1000, max=2000, step=100, description="件数:",)
slider2.observe(store_upper, names="value")
display(slider2)

IntSlider(value=1600, description='件数:', max=2000, min=1000, step=100)

### ノック２４：トグルボタンで地域データを抽出しよう

### ノック２５：日付を指定してデータを抽出してみよう

### ノック２６：ストーリーを考えてデータを構築しよう

### ノック２７：キャンセルの理由を分析してみよう

### ノック２８：仮説を検証してみよう

### ノック２９：ストーリーをもとにパーツやデータを組み合わせてダッシュボードを作ろう

### ノック３０：ダッシュボードを改善しよう